In [ ]:
import nltk
import pandas as pd

In [ ]:
#nltk.download_shell()

In [ ]:
#messages=[line.rstrip() for line in open('smsspamcollection/SMSSpamCollection')]

In [ ]:
messages = pd.read_csv('../input/spam.csv',encoding='latin-1')

In [ ]:
messages.head(2)

In [ ]:
messages.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace = True)

In [ ]:
import pandas as pd
column_names=['label','msg']

In [ ]:
messages.columns = column_names

In [ ]:
messages.head()

In [ ]:
messages.groupby('label').describe()

In [ ]:
messages['length']=messages['msg'].apply(len)

In [ ]:
messages.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
plt.figure(figsize=(10,4))
messages['length'].plot.hist(bins=200)

In [ ]:
messages['length'].describe()

OMG such a big text, let's check it out.... ;D

In [ ]:
messages[messages['length']==910]['msg'].iloc[0]

Ooops... Wrong decision...

In [ ]:
messages.hist(column='length',by='label',bins=100,figsize=(10,4))

In [ ]:
import string

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords.words('english')

Time to take a leap of faith
performing "removal of punctuation" {return a char if char is not in a set of string.punctuation}
perform " Joining of words"
perform "removal of stop words"{return char if lower version of char is not in stop words}

In [ ]:
def text_process(mess):
    '''
    1.remove punc
    2.remove stop words
    3.return clean text
    '''
    nopunc=[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
messages.head()

In [ ]:
messages['msg'].head().apply(text_process)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer=CountVectorizer(analyzer=text_process).fit(messages['msg'])

In [ ]:
print(len(bow_transformer.vocabulary_))

In [ ]:
mess4=messages['msg'][3]

In [ ]:
mess4

In [ ]:
bow_transformer

In [ ]:
bow4=bow_transformer.transform([mess4])

In [ ]:
print(bow4)

In [ ]:
bow_transformer.get_feature_names()[4551]

In [ ]:
type(bow_transformer)

In [ ]:
messages_bow=bow_transformer.transform(messages['msg'])

In [ ]:
print('shape of sparse matrix:',messages_bow.shape)

In [ ]:
messages_bow.nnz

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer=TfidfTransformer().fit(messages_bow)

In [ ]:
tfidf4=tfidf_transformer.transform(bow4)

In [ ]:
print(tfidf4)

In [ ]:
tfidf_transformer.idf_[bow_transformer.vocabulary_['university']]

In [ ]:
messages_tfidf = tfidf_transformer.transform(messages_bow)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
spam_detect_model = MultinomialNB().fit(messages_tfidf,messages['label'])

In [ ]:
spam_detect_model.predict(tfidf4)

In [ ]:
all_pred=spam_detect_model.predict(messages_tfidf)

In [ ]:
all_pred

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
msg_train,msg_test,label_train,label_test=train_test_split(messages['msg'],messages['label'],test_size=0.3)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
])

In [ ]:
pipeline.fit(msg_train,label_train)

In [ ]:
predictions = pipeline.predict(msg_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(label_test,predictions))